In [ ]:
import json
from pathlib import Path

import pandas as pd

# sample_idx, exp, test_id, fluency, relevance, logical_soundness, normal




In [ ]:
RESULT_DIR = Path("/data/vtt/human_evaluation")
BAD_SAMPLE_IDX = [1336, 695, 23, 929]

files = RESULT_DIR.glob("*.json")
results = []
for file in files:
    with open(file, "r") as f:
        data = json.load(f)
    if data["sample_idx"] in BAD_SAMPLE_IDX:
        continue
    try:
        results.append({
            "sample_idx": data["sample_idx"],
            "exp": data["exp"],
            "test_id": int(file.stem.split("_")[-1]),
            "fluency": int(data["fluency"]),
            "relevance": int(data["relevance"]),
            "logical_soundness": int(data["logical_soundness"]),
            "normal": data["normal"]
        })
    except:
        print(file)
df = pd.DataFrame(results)
df = df.sort_values(by=["sample_idx", "exp"])

In [ ]:
from scipy import stats

In [ ]:
exps = df.exp.unique()
results_dict = {}
for exp in exps:
    results_dict[exp] = df[df.exp==exp]

In [ ]:
stats.ttest_ind(results_dict['ttnet_base'].logical_soundness, results_dict['ttnet'].logical_soundness)

In [ ]:
results_table = []
df_ttnet = df[df.exp=="ttnet"]
def significant(df_exp, key):
    _, p_value = stats.ttest_ind(df_ttnet[key], df_exp[key])
    return "$\dagger$" if p_value < 0.05 else ""
for exp in exps:
    df_exp = df[df.exp==exp]
    mean = df_exp.mean()
    std = df_exp.std()
    results_table.append({
        "exp": exp,
        "fluency": f"{mean['fluency']:.2f}{significant(df_exp, 'fluency')}",
        "relevance": f"{mean['relevance']:.2f}{significant(df_exp, 'relevance')}",
        "LS": f"{mean['logical_soundness']:.2f}{significant(df_exp, 'logical_soundness')}"
        # "fluency": f"{mean['fluency']:.2f} {std['fluency']:.2f}",
        # "relevance": f"{mean['relevance']:.2f} {std['relevance']:.2f}",
        # "LS": f"{mean['logical_soundness']:.2f} {std['logical_soundness']:.2f}"
    })

In [ ]:
print(pd.DataFrame(results_table).style.to_latex())

In [ ]:
df[df.exp=="ttnet"].var()

In [ ]:
df[df.exp=="ttnet_base"].mean()

In [ ]:
df[df.exp=="glacnet"].mean()

In [ ]:
df[df.exp=="densecap"].mean()

In [ ]:
df[df.exp=="cst"].mean()